In [4]:
import os
import shutil

# Make .kaggle folder
os.makedirs("/root/.kaggle", exist_ok=True)




In [ ]:
!pip install ultralytics -q

from ultralytics import YOLO
import os
import shutil

# STOP: First, let's check what's actually happening with the symbolic links
print("Checking symbolic links...")
try:
    ls_output = !ls -la /kaggle/working/fixed_dataset/images/
    print("Images folder:", ls_output)
    
    ls_output = !ls -la /kaggle/working/fixed_dataset/labels/
    print("Labels folder:", ls_output)
except:
    print("Symbolic links may not have worked properly")

# BETTER SOLUTION: COPY THE ACTUAL FILES to proper structure
print("\nCopying files to proper structure...")

# Create proper folder structure
os.makedirs('/kaggle/working/proper_dataset/images/train', exist_ok=True)
os.makedirs('/kaggle/working/proper_dataset/labels/train', exist_ok=True)

# Copy images
src_images = "/kaggle/input/pipeline-defect-dataset/images/images/train"
dst_images = "/kaggle/working/proper_dataset/images/train"

image_files = [f for f in os.listdir(src_images) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"Copying {len(image_files)} images...")
for img_file in image_files[:100]:  # Copy first 100 to test
    shutil.copy2(os.path.join(src_images, img_file), dst_images)

# Copy labels
src_labels = "/kaggle/input/pipeline-defect-dataset/labels/labels/train"
dst_labels = "/kaggle/working/proper_dataset/labels/train"

label_files = [f for f in os.listdir(src_labels) if f.endswith('.txt')]
print(f"Copying {len(label_files)} labels...")
for label_file in label_files[:100]:  # Copy first 100 to test
    shutil.copy2(os.path.join(src_labels, label_file), dst_labels)

print("Files copied successfully!")

# Create CORRECT config file
corrected_config_content = """
path: /kaggle/working/proper_dataset
train: images/train
val: images/train
nc: 6
names: 
  0: Deformation
  1: Obstacle
  2: Rupture
  3: Disconnect
  4: Misalignment
  5: Deposition
"""

with open('/kaggle/working/corrected_config.yaml', 'w') as f:
    f.write(corrected_config_content)

print("Created corrected_config.yaml")

# Verify the files are in the right place
print("\nVerifying file structure:")
print("Images in train:", len(os.listdir('/kaggle/working/proper_dataset/images/train')))
print("Labels in train:", len(os.listdir('/kaggle/working/proper_dataset/labels/train')))

# Load a pretrained model
model = YOLO("yolov8n.pt")  

# Train using the PROPER dataset structure
model.train(
    data="/kaggle/working/corrected_config.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    workers=2,
    project="/kaggle/working",
    name="pipeline_defect_yolov8_proper",
    save=True,
)

print("Training started with proper file structure!")

In [ ]:
import shutil
import os

# Compress the entire training directory
directory_to_download = "/kaggle/working/pipeline_defect_yolov8_proper"
zip_output_path = "/kaggle/working/pipeline_defect_model_complete"

if os.path.exists(directory_to_download):
    # Create a zip file of the entire directory
    shutil.make_archive(zip_output_path, 'zip', directory_to_download)
    print("✅ Directory compressed successfully!")
    print(f"📦 Zip file created: {zip_output_path}.zip")
    print("\n📥 Download instructions:")
    print("1. Go to right sidebar → 'Data' → 'Output'")
    print("2. Find 'pipeline_defect_model_complete.zip'")
    print("3. Click the download icon 📁 next to it")
else:
    print("❌ Directory not found! Check the path.")